In [28]:
import math
import numpy as np
import pandas as pd

#Ler conjunto de dados de preenchimento
normal = open("./normal/normal-perfil-3g.csv").read().splitlines()
bufferoverflow = open('./perfil_3g/bufferoverflow-perfil.csv').read().splitlines()
crlfi = open('./perfil_3g/crlfi-perfil.csv').read().splitlines()
formatstring = open('./perfil_3g/formatstring-perfil.csv').read().splitlines()
ldapi = open('./perfil_3g/ldapi-perfil.csv').read().splitlines()
sqli = open('./perfil_3g/sqli-perfil.csv').read().splitlines()
ssi = open('./perfil_3g/ssi-perfil.csv').read().splitlines()
xpath = open('./perfil_3g/xpath-perfil.csv').read().splitlines()
xss = open('./perfil_3g/xss-perfil.csv').read().splitlines()

print('normal=', len(normal))
print('bufferoverflow=', len(bufferoverflow))
print('crlfi=', len(crlfi))
print('formatstring=', len(formatstring))
print('ldapi=', len(ldapi))
print('sqli=', len(sqli))
print('xpath=', len(xpath))
print('xss=', len(xss))

normal= 8161
bufferoverflow= 412
crlfi= 327
formatstring= 41
ldapi= 70
sqli= 18355
xpath= 166
xss= 4816


In [29]:
from bitarray import bitarray
import mmh3

class BloomFilter:
    
    def __init__(self, size, hash_count):
        self.size = size
        self.hash_count = hash_count
        self.bit_array = bitarray(size)
        self.bit_array.setall(0)
        
    def add(self, string):
        for seed in range(self.hash_count):
            result = mmh3.hash(string, seed) % self.size
            self.bit_array[result] = 1
            
    def lookup(self, string):
        for seed in range(self.hash_count):
            result = mmh3.hash(string, seed) % self.size
            if self.bit_array[result] == 0:
                return False # Se 0 é pq não está no array
        return True # Se 1 é pq talvez está no array

In [30]:
#Teste com os dados normais
def filtro(grupo):
    dados = []
    a = []
    for i in grupo:
        cont = 0
        tem = 0
        nao_tem = 0
        for j in i:
            if bf.lookup(j) == True:
                tem +=1
            else:
                nao_tem +=1
        a = tem, nao_tem
        dados.append(a)
    return dados

In [31]:
# Escolhe a base de treino
treino = normal

n = len(treino)
p = 0.01

#Calcular tamanho do Filtro
m = -(n*math.log(p))/(math.log(2)**2)
k = (m/n) * math.log(2)

# Arredonda o valores de m e k
m_tam = round(m)
k_hash = round(k)

# inicia o filtro com os parametros padrões
bf = BloomFilter(m_tam, 9)

# preenche o filtro com os dados de treino
for i in treino:
    bf.add(i)

# Imprime informações úteis
print("Qtde de itens na base de treino:", n)
print("Probabilidade de FP:", p)
print("Tamanho do Filtro:", m_tam)
print("Nº de funções hash:", k_hash)

Qtde de itens na base de treino: 8161
Probabilidade de FP: 0.01
Tamanho do Filtro: 78224
Nº de funções hash: 7


In [32]:
def consultaFiltro(perfis):
    VP = 0
    FN = 0
            
    for i in perfis:
        if bf.lookup(i) == False:
            VP +=1
        else:
            FN +=1
    
    TD = VP/(VP+FN)
    return VP, FN, TD

In [34]:
resultado = []

resultado.append(consultaFiltro(bufferoverflow))
resultado.append(consultaFiltro(crlfi))
resultado.append(consultaFiltro(formatstring))
resultado.append(consultaFiltro(ldapi))
resultado.append(consultaFiltro(sqli))
resultado.append(consultaFiltro(xpath))
resultado.append(consultaFiltro(xss))
resultado.append(consultaFiltro(normal))

resultado
df = pd.DataFrame(np.array(resultado).reshape(8,3), columns = list(['VP','FN','TD']), )
df

,VP,FN,TD
0,411.0,1.0,0.997573
1,327.0,0.0,1.000000
2,41.0,0.0,1.000000
3,70.0,0.0,1.000000
4,18335.0,20.0,0.998910
5,166.0,0.0,1.000000
6,4812.0,4.0,0.999169
7,0.0,8161.0,0.000000
